# Bitcoin from scratch

Bitcoin is a software program that runs on a decentralized network of computers. The software defines the rules of Bitcoin, including how transactions are verified and how new bitcoins are created.
Bitcoin uses cryptography to secure transactions and to prevent fraud. Cryptography is the science of encrypting and decrypting data. In Bitcoin, cryptography is used to create digital signatures, which are used to verify the authenticity of transactions.

Bitcoin is a decentralized currency, meaning that it is not controlled by any central authority. The Bitcoin network is run by a network of volunteers around the world.
All Bitcoin transactions are recorded on the blockchain, which is a public ledger. This means that anyone can view the history of Bitcoin transactions.

The purpose of this notebook is to discover and analyze as deeply as possible the various components of Bitcoin. The final goal is to create, digitally sign, and broadcast a Bitcoin transaction.

By the end of this notebook, you will have a better understanding of how a Bitcoin transaction works.
You will be able to create, digitally sign, and broadcast your own Bitcoin transactions. You will also be able to understand the security and privacy implications of Bitcoin.

Here is a roadmap of the notebook:

1) Introduction to the cryptography behind Bitcoin
- Concept of Finite field
- Modular arithmetic
- Elliptic Curve Cryptography (ECC)
2) Generation of the Crypto identity
- Generation of the Private key
- Generate the Public key from the Private key
- Generation of the Bitcoin address


## Introduction to the cryptography behind Bitcoin

### Finite fields

Finite fields, also known as Galois fields, are fundamental mathematical structures used in various fields, including algebra, coding theory, cryptography, and computer science.

A field is a mathematical structure that consists of a set of elements along with two binary operations: addition and multiplication, which satisfy certain properties.
Specifically, a field must satisfy the following conditions:
- Closure: The result of addition or multiplication of any two elements in the field is also an element in the field.
- Associativity: Addition and multiplication operations are associative.
- Commutativity: Both addition and multiplication are commutative; the order of operands does not matter.
- Identity Elements: There exist unique elements called the additive and multiplicative identities (denoted as 0 and 1, respectively) such that adding or multiplying any element by these identities leaves the element unchanged.
- Inverse Elements: Every nonzero element in the field has a unique additive and multiplicative inverse.
- Distributive Property: Multiplication distributes over addition.

A finite field is a field that has a finite number of elements. The order of a finite field is the number of elements it contains. For any finite field, the order is a prime power, represented as "q," where "q" is a prime number raised to a positive integer power.

The characteristic of a finite field is the smallest positive integer "p" such that summing "p" copies of the multiplicative identity (1) yields the additive identity (0). The characteristic can be either a prime number or zero. If the characteristic is zero, the field is said to have characteristic zero.

Finite fields can be constructed using various methods, but the most common method is through polynomial construction. For each prime power "q," there exists a unique (up to isomorphism) finite field of order "q." This finite field is denoted as GF(q) or Fq. The elements of a finite field Fq are usually denoted as 0, 1, 2, ..., q-1.

Finite fields can have subfields, which are smaller fields contained within the original finite field. These subfields can also be finite fields themselves. Additionally, one can create extensions of finite fields by defining new finite fields with larger orders.

For each prime power "q," there is a unique finite field of order "q" up to isomorphism. This means that any two finite fields of the same order are essentially the same in terms of their algebraic properties, even though they might be represented with different sets of elements.

The study of finite fields is closely related to Galois theory, a branch of abstract algebra. Galois theory explores the relationship between field extensions and symmetries of polynomial equations. Finite fields are an important example of finite field extensions in Galois theory.

In [ ]:
# Some imports
from __future__ import annotations # https://peps.python.org/pep-0563/ PEP 563 – Postponed Evaluation of Annotations
from dataclasses import dataclass # https://docs.python.org/3/library/dataclasses.html This module provides a decorator and functions for automatically adding generated special methods such as __init__() and __repr__() to user-defined classes.


In [1]:
# Constructing a finite field
class FieldElement:

    def __init__(self, num, prime):
        if num >= prime or num < 0:
            error = 'Num {} not in field range 0 to {}'.format(num, prime - 1)
            raise ValueError(error)
        self.num = num
        self.prime = prime

    def __repr__(self):
        return 'FieldElement_{}({})'.format(self.prime, self.num)

    # Equal
    def __eq__(self, other):
        if other is None:
            return False
        return self.num == other.num and self.prime == other.prime

    # Not equal
    def __ne__(self, other):
        return not (self == other)

    # Addition
    def __add__(self, other):
        if self.prime != other.prime:
            raise TypeError('Cannot add two numbers in different fields')
        num = (self.num + other.num) % self.prime
        return self.__class__(num, self.prime)

    # Subtraction
    def __sub__(self, other):
        if self.prime != other.prime:
            raise TypeError('Cannot subtract two numbers in different fields')
        num = (self.num - other.num) % self.prime
        return self.__class__(num, self.prime)

    # Multiplication
    def __mul__(self, other):
        if self.prime != other.prime:
            raise TypeError('Cannot multiply two numbers in different fields')
        num = (self.num * other.num) % self.prime
        return self.__class__(num, self.prime)

    # Division
    # use the Fermat's little theorem:
    # self.num**(p-1) % p == 1
    # that means 1/n == pow(n, p-2, p)
    def __truediv__(self, other):
        if self.prime != other.prime:
            raise TypeError('Cannot divide two numbers in different Fields')
        num = self.num * pow(other.num, self.prime - 2, self.prime) % self.prime
        return self.__class__(num, self.prime)

    # redefining the pow function
    def __pow__(self, exponent):
        n = exponent % (self.prime-1)
        num = pow(self.num, n, self.prime)
        return self.__class__(num, self.prime)


In [2]:
a = FieldElement(7, 13)
b = FieldElement(6, 13)

print(a == b)   # False


False


### Modulo arithmetic

Modulo arithmetic, also known as clock arithmetic or modular arithmetic, is a fundamental branch of arithmetic that deals with numbers' remainders after division. It is a way of performing arithmetic operations on integers that are constrained to a fixed positive integer, known as the modulus.

In the context of modulo arithmetic, let's consider two integers: a (the dividend) and m (the modulus), where m is a positive integer greater than 1. The symbol used to denote the modulo operation is "mod." The result of the modulo operation is the remainder obtained when dividing 'a' by 'm.'

Mathematically, the modulo operation is represented as:

a mod m = r

Here, 'r' represents the remainder obtained when 'a' is divided by 'm.'

Key properties of modulo arithmetic:

Range constraint: The result of the modulo operation is always within the range of 0 to (m - 1). In other words, 0 ≤ (a mod m) ≤ (m - 1).

Congruence: If two integers 'a' and 'b' have the same remainder when divided by 'm,' they are said to be congruent modulo 'm,' represented as a ≡ b (mod m). This means (a mod m) = (b mod m).

Common arithmetic operations in modulo arithmetic:

Addition: To perform addition in modulo arithmetic, simply add the two numbers and take the result modulo 'm.'

(a + b) mod m = (a mod m + b mod m) mod m

Subtraction: To perform subtraction, similarly subtract the two numbers and take the result modulo 'm.'

(a - b) mod m = (a mod m - b mod m) mod m

Multiplication: For multiplication, multiply the two numbers and then take the result modulo 'm.'

(a * b) mod m = (a mod m * b mod m) mod m

Exponentiation: For exponentiation, raise 'a' to the power of 'b' and take the result modulo 'm.'

(a^b) mod m = (a mod m)^b mod m

Modulo arithmetic finds numerous applications in various fields, including computer science, cryptography, number theory, and digital signal processing. In computer science, it is used in hashing functions, random number generation, and cyclic data structures. In cryptography, it forms the basis of some encryption and decryption algorithms.

Modulo arithmetic has a cyclical nature, akin to a clock that repeats itself after completing a full circle. This property makes it a valuable tool in solving problems involving periodicity and repetitions.


In [5]:
# Addition on a finite field

a = FieldElement(7, 19)
b = FieldElement(8, 19)
c = FieldElement(15, 19)

# (7 + 8) % 19 = 15
print(a+b==c) # True

# Multiplication on a finite field
a = FieldElement(3, 13)
b = FieldElement(12, 13)
c = FieldElement(10, 13)

print(a*b==c) # True

# Exponentiation on a finite field
a = FieldElement(3, 13)
b = FieldElement(1, 13)

print(a**3==b) # True

# Division on a finite field
"""
In normal math, division is the inverse of multiplication
7 x 8 = 56 implies that 56 / 8 = 7
12 x 2 = 24 implies that 24 / 12 = 2
Dividing any two numbers where the denominator is not 0 will result in another finite field element.
n^(p-1) is always is 1 for every p that is prime and every n>0. This comes from number theory called Fermat's little theorem.
n^(p-1)%p=1
where p is prime.
{1 x 2 x 3 ... x (p-2) x (p-1) % p = n x 2n x 3n x ... x (p-2)n x (p-1)n % p}
"""

a = FieldElement(3, 31)
b = FieldElement(24, 31)
c = FieldElement(4, 31)

print(a/b==c) # True


True
True
True
True


### Elliptic Curve Cryptography

Elliptic Curve Cryptography (ECC) is a type of asymmetric or public key cryptography based on the algebraic structures of the elliptic curves over finite fields and on the discrete logarithm problem as expressed by addition and multiplication on the points of the curve.

The foundation of ECC lies in the mathematical properties of elliptic curves. An elliptic curve is a smooth curve defined by an equation of the form:

y^2 = x^3 + ax + b

'a' and 'b' are constants that define the specific curve's shape, and the coordinates 'x' and 'y' represent points on the curve. However, these coordinates must satisfy certain conditions. One key property of elliptic curves is that they have an additive group structure, allowing for point addition and scalar multiplication operations.

ECC uses this group structure to create cryptographic key pairs: a private key and a corresponding public key. The private keys in the ECC (usually denoted as 'd') are integers (in the range of the curve's field size, tipically 256-bit integers).
The key generation in the ECC cryptography is as simple as securely generating a random integer in certain range, so any number within the range is a valid ECC private key.

The public key is a point on the elliptic curve obtained by multiplying the generator point (a specific, fixed point on the curve) by the private key

Public key (P) = Private key (d) * Generator point (G)

ECC crypto algorithms can use different underlying elliptic curves. Different curves provide different levels of security, performance and key length.

A point at infinity is needed to be part of the curve, and this point can be denoted as 0 (zero).

The ECC uses elliptic curves over the finite field Fp, where p is prime and p > 3.
This means that the field is a square matrix of size p x p and the points on the curve are limited to integer coordinates within the field only.

ECC is employed in two primary cryptographic functions: key exchange and digital signatures.

1) Key Exchange:
ECC allows two parties to agree on a shared secret key over an insecure channel securely. The parties exchange their public keys, perform point multiplication using their private keys and the received public keys, and arrive at the same shared secret point. This point is then used to derive a symmetric encryption key or to initialize a secure communication session.

2) Digital Signatures:
In ECC-based digital signatures, the private key holder signs a message using their private key to produce a unique signature. The signature can be publicly verified using the corresponding public key. Verifying the signature involves performing point operations to ensure the authenticity and integrity of the message.

Overall, Elliptic Curve Cryptography offers strong security with relatively smaller key sizes compared to traditional cryptographic algorithms, making it a popular choice for various applications in the modern digital landscape, including secure communications, digital signatures, and secure key exchange protocols.
